[Reference](https://medium.com/@Divithraju/event-oriented-web-scraping-with-python-and-kafka-6f37267818f6)

In [2]:
pip install kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 11.9 MB/s eta 0:00:00


In [4]:
from kafka import KafkaProducer
from bs4 import BeautifulSoup
import requests
import time

# Kafka configuration
bootstrap_servers = ['localhost:9092']
topic = 'scraping-events'

# Web scraper configuration
product_url = 'https://www.domain.com/product-page'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# Create Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

def scrape_product_price():
    # Make HTTP request to fetch the product page
    response = requests.get(product_url, headers=headers)

    # Parse the HTML response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the product price from the parsed HTML
    price_element = soup.find('span', {'class': 'product-price'})
    price = price_element.text.strip() if price_element else 'N/A'

    # Create a scraping event message
    scraping_event = {
        'timestamp': int(time.time()),
        'product_url': product_url,
        'price': price
    }

    # Publish the scraping event to Kafka topic
    producer.send(topic, value=scraping_event)
    producer.flush()
    print('Scraping event published:', scraping_event)

# Run the scraper at regular intervals
while True:
    scrape_product_price()
    time.sleep(300)
# Scrapes every 5 minutes